# Teleoperated Interactive Regression

In this example we will use gamepad to control the robot in order to capture drive commands with associated images.

### Create gamepad controller and connect it to the robot - based on teleoperation.ipynb

First we create a gamepad controller. 

In [1]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

Next, we create a robot class.

> WARNING: This next cell will move the robot if you touch the gamepad controller axes!

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar
import traitlets

car = NvidiaRacecar()

In [3]:
car.throttle_gain   = 0.3
car.steering_offset = 0.05
car.steering        = 0.0

In [4]:
# this cell tends to fail when called immediately after the previous (tuple index out of range). When it happens - rerun the cell. Ah Python... ;]
steering_link  = traitlets.dlink((controller.axes[0], 'value'), (car, 'steering'), transform=lambda x:-x)
throttle_link = traitlets.dlink((controller.axes[3], 'value'), (car, 'throttle'), transform=lambda x: x)

Now we create a camera to provide a feedback. Notice the reduced framerate. We really do not need 65 FPS. Lower framerate is much better from training perspective so that we do not have multiple data that is not distinctively different. 

In [5]:
from jetcam.csi_camera import CSICamera
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_fps=5)
image = camera.read()
image_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

image_widget.value = bgr8_to_jpeg(image)

display(image_widget)
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

As we do not like wasting processing power, make sure camera is running only when driving forward. Furthermore, capturing dataset while stationary could result in overfit. Use backward motion to reposition robot where you want.
> NOTE: In this case, negative X value is forward motion.

In [6]:
camera_run_link = traitlets.dlink((car, 'throttle'), (camera, 'running'), transform=lambda x: x < 0)

This is a good time to test if the robot is moving correctly and camera widget is updating. If needed, adjust steering offset and throttle_gain.

### Now it is time to create ML part of the example - based on interactive_regression.ipynb

Task - copy-paste, only task name and categories are renamed.

In [7]:
import torchvision.transforms as transforms
from xy_dataset_float import XYDataset_Float

TASK = 'road_following_teleoperated'

CATEGORIES = ['apex_2']

DATASETS = ['A', 'B']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset_Float(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)

Data Collection - note the modifications compared to interactive_regression.ipynb

In [8]:
# initialize active dataset
dataset = datasets[DATASETS[0]]

# we do not touch the camera class as it has been already initialised and preview widget was created. Besides, we don't really need a preview.

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.unobserve_all()
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.unobserve_all()
category_widget.observe(update_counts, names='value')

# this function is only called when new image is acquired.
def saveData(change):
    # get the robot's steering and throttle instead of X-Y pixel coordinates
    x = car.steering
    y = car.throttle
    # save to disk
    dataset.save_entry(category_widget.value, camera.value, x, y)
    count_widget.value = dataset.get_count(category_widget.value)
image_widget.unobserve_all()
image_widget.observe(saveData, names = 'value')

data_collection_widget = ipywidgets.VBox([
    image_widget,
    dataset_widget,
    category_widget,
    count_widget
])

display(data_collection_widget)

Model - copy-paste from interactive_regression.ipynb

In [9]:
import torch
import torchvision

device = torch.device('cuda')
output_dim = 2 * len(dataset.categories)  # x, y coordinate for each category

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# DENSENET 121
# model = torchvision.models.densenet121(pretrained=True)
# model.classifier = torch.nn.Linear(model.num_features, output_dim)

model = model.to(device)

model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value='road_following_model_tele.pth')

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])

display(model_widget)

Train

In [11]:
import time

BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()

        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, category_idx, xy in iter(train_loader):
                # send data to device
                images = images.to(device)
                xy = xy.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute MSE loss over x, y coordinates for associated categories
                loss = 0.0
                for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                    loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
                loss /= len(category_idx)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                count = len(category_idx.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except Exception as e:
        print(e.message, e.args)
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

display(train_eval_widget)